In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes

Load a dataset

In [2]:
df=pd.read_csv('C:/Users/Arup/Downloads/diabetes.csv')
df.head(10)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
5,-0.092695,-0.044642,-0.040696,-0.019442,-0.068991,-0.079288,0.041277,-0.076395,-0.041180,-0.096346,97.0
6,-0.045472,0.050680,-0.047163,-0.015999,-0.040096,-0.024800,0.000779,-0.039493,-0.062913,-0.038357,138.0
7,0.063504,0.050680,-0.001895,0.066630,0.090620,0.108914,0.022869,0.017703,-0.035817,0.003064,63.0
8,0.041708,0.050680,0.061696,-0.040099,-0.013953,0.006202,-0.028674,-0.002592,-0.014956,0.011349,110.0
9,-0.070900,-0.044642,0.039062,-0.033214,-0.012577,-0.034508,-0.024993,-0.002592,0.067736,-0.013504,310.0


In [3]:
df=pd.read_csv('C:/Users/Arup/Downloads/diabetes.csv')
df1 = load_diabetes()
df = pd.DataFrame(df1['data'], columns=df1.feature_names)
df['target'] = pd.DataFrame(df1['target'])
df.to_csv('C:/Users/Arup/Downloads/diabetes.csv', index=False)

Split into train and test data
train data:80%
test data: 20%

In [4]:
train_df=df.sample(frac=0.8,random_state=42)
test_df=df.drop(train_df.index)

Determine corelation of each features with the target and remove features which correlation is between -0.2 and 0.2

In [5]:
correlation_matrix=train_df.drop('target',axis=1).apply(lambda x: x.corr(train_df['target'],method='pearson'))
print(correlation_matrix)
drop_x=[]
ind=0
for i in correlation_matrix:
    
    if abs(i)<=0.2:
        drop_x.append(correlation_matrix.index[ind])
    ind=ind+1
train_df=train_df.drop(drop_x,axis=1)
test_df=test_df.drop(drop_x,axis=1)

age    0.215813
sex    0.050006
bmi    0.589735
bp     0.427953
s1     0.206135
s2     0.164852
s3    -0.433796
s4     0.446553
s5     0.591447
s6     0.399071
dtype: float64


In [6]:
train_df.drop('target',axis=1).corr(method='pearson')

,age,bmi,bp,s1,s3,s4,s5,s6
age,1.000000,0.237226,0.352439,0.285912,-0.106037,0.234188,0.280950,0.337250
bmi,0.237226,1.000000,0.405839,0.238590,-0.381233,0.411264,0.444145,0.379617
bp,0.352439,0.405839,1.000000,0.235827,-0.206316,0.268278,0.393532,0.391683
s1,0.285912,0.238590,0.235827,1.000000,0.005237,0.573142,0.510195,0.316144
s3,-0.106037,-0.381233,-0.206316,0.005237,1.000000,-0.743655,-0.435459,-0.296656
s4,0.234188,0.411264,0.268278,0.573142,-0.743655,1.000000,0.619710,0.433250
s5,0.280950,0.444145,0.393532,0.510195,-0.435459,0.619710,1.000000,0.465463
s6,0.337250,0.379617,0.391683,0.316144,-0.296656,0.433250,0.465463,1.000000


scaling dataste: new range of dataset [0,1]

In [7]:
train_df_scale=(train_df-train_df.min())/(train_df.max()-train_df.min())

test_df_scale=(test_df-train_df.min())/(train_df.max()-train_df.min())

In [8]:
train_df_scale.insert(0,'Ones', 1)
train_X=train_df_scale.drop('target', axis=1)
train_y=train_df_scale['target']

test_df_scale.insert(0,'Ones', 1)
test_X=test_df_scale.drop('target', axis=1)
test_y=test_df_scale['target']

Determining constant and coefficients from train data. 

In [9]:
beta=np.linalg.inv(train_X.T.dot(train_X)).dot(train_X.T).dot(train_y)
print(beta)

[-0.005568   -0.01423983  0.44375958  0.16494986 -0.22468898 -0.05431699
  0.0994972   0.50279287  0.06569928]


In [10]:
test_y_pred=pd.DataFrame(test_X.dot(beta))

Determing MSE and MAE for the test data

In [11]:
MSE=np.mean((test_y_pred-[test_y])**2)

MAE=np.mean(abs(test_y_pred-[test_y]))

Converting scaled value of target variable into actual value

In [12]:
test_y_pred_true=test_y_pred*(train_df['target'].max()-train_df['target'].min())+train_df['target'].min()